In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import urllib

In [4]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')

In [5]:
def search_info(string):
    try:
        # Поиск названия фильма и режиссера
        film_info = re.search(r'(?<=Directed by )(.+?)(?=\.)', string)
        if film_info:
            director = film_info.group(1).strip()

        # Поиск главных актеров и описания фильма
        actors = re.findall(r"With (.*?\.)", string)[0]

        description = re.findall(rf"{actors}(.*)", string, flags=re.DOTALL)[0].strip()

        return(director,actors[:-1],description)
    except:
        return(np.NaN,np.NaN,np.NaN)


In [6]:
def get_movie_description(id):
    # url в зависимости от длины id
    try:
        if id > 999999:
            url = f'https://www.imdb.com/title/tt{id}/'
        elif 99999<id<1000000:
            url = f'https://www.imdb.com/title/tt0{id}/'
        elif 9999<id<100000:
            url = f'https://www.imdb.com/title/tt00{id}/'
        
        hdr = {'User-Agent': 'Chrome/19.0'}

        req = urllib.request.Request(url, headers=hdr)
        page = urllib.request.urlopen(req, timeout=60)

        # сознаю объект BeautifulSoup для парсинга страницы
        soup = BeautifulSoup(page, 'html.parser')

        # поиск тега <meta>
        meta_tag = soup.find("meta", {"name": "description"})

        # если тег найден, возвращаем его содержимое
        if meta_tag:
            return meta_tag['content']
        return ''
    except:
        return ''

In [12]:
# заполним таблицу info
info = links.copy()
for n,id in tqdm(enumerate(links['imdbId'])):
    full_string = get_movie_description(id)  
    full_info = search_info(full_string)

    info.loc[info['imdbId'] == id, 'director'] =full_info[0]
    info.loc[info['imdbId'] == id, 'actors'] = full_info[1]
    info.loc[info['imdbId'] == id, 'description'] = full_info[2]
    info.to_csv('info.csv')


41it [01:47,  2.62s/it]


In [18]:
info = pd.read_csv('info.csv')

In [30]:
# поправлю те имена актёров, что плохо считались (инициалы, Jr.)
for i, row in info.iterrows():
    if row['actors']==row['actors'] and (len(row['actors']) == 1 or (row['actors'][-2] == ' ' and len(row['actors']) > 1) or (row['actors'][-2:] == 'Jr')):
        desc = row['description'].split('.')[0].strip()
        info.at[i, 'actors'] += '. ' + desc
        info.at[i, 'description'] = row['description'][len(desc)+1:].strip()

In [34]:
# если плохо считался директор
for i, row in info.iterrows():
    if row['director'] == row['director'] and (
        len(row['director']) == 1 or (row['director'][-2] == ' ' and len(row['director']) > 1) or (row['director'][-2:] == 'Jr')
    ):
        movie_description = get_movie_description(row['imdbId'])
        
        start = movie_description.find('Directed by') + len('Directed by')
        end = movie_description.find('With', start)
        director_name = movie_description[start:end].strip()
        
        info.at[i, 'director'] = director_name

In [36]:
for i, row in info.iterrows():
    if row['director'] == row['director'] and (len(row['director']) > 1) and (row['director'][-1] == '.'):
        info.at[i, 'director'] = row['director'][:-1]

In [ ]:
info['genres'] = movies['genres'].str.replace('|', ', ')

In [38]:
info = info.drop(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1','Unnamed: 0'], axis=1)

info.to_csv('info.csv')

In [10]:
info.to_csv('info.csv')